# XHEC - Session 5-1


-----------------

## Topic Extraction : LDA Implementation  

In this session we will build an LDA from scratch

### Import libraries 

In [ ]:
import os
import pandas as pd
import numpy as np
import itertools
import random

### Create documents 

In [ ]:
rawdocs = ['eat turkey on turkey day holiday',
           'i like to eat cake on holiday',
           'turkey trot race on thanksgiving holiday',
           'snail race the turtle',
           'time travel space race',
           'movie on thanksgiving',
           'movie at air and space museum is cool movie',
           'aspiring movie star']

rawdocs = list(map(lambda x: x.split(), rawdocs)) #Split by whitespace

In [ ]:
rawdocs

[['eat', 'turkey', 'on', 'turkey', 'day', 'holiday'],
 ['i', 'like', 'to', 'eat', 'cake', 'on', 'holiday'],
 ['turkey', 'trot', 'race', 'on', 'thanksgiving', 'holiday'],
 ['snail', 'race', 'the', 'turtle'],
 ['time', 'travel', 'space', 'race'],
 ['movie', 'on', 'thanksgiving'],
 ['movie', 'at', 'air', 'and', 'space', 'museum', 'is', 'cool', 'movie'],
 ['aspiring', 'movie', 'star']]

### Set parameters

In [ ]:
K = 2 #Number of topic
alpha = 0.1 #Hyperparameter alpha
eta = 0.1 #Hyperparameter eta
iterationNb = 3 #Number of iterations

### Convert to a numerical problem 

In [ ]:
#Create a dictionnary {id: word}
vocab = np.unique(list(itertools.chain.from_iterable(rawdocs)))
vocab = {k: v for v, k in enumerate(vocab)}

In [ ]:
vocab

{'air': 0,
 'and': 1,
 'aspiring': 2,
 'at': 3,
 'cake': 4,
 'cool': 5,
 'day': 6,
 'eat': 7,
 'holiday': 8,
 'i': 9,
 'is': 10,
 'like': 11,
 'movie': 12,
 'museum': 13,
 'on': 14,
 'race': 15,
 'snail': 16,
 'space': 17,
 'star': 18,
 'thanksgiving': 19,
 'the': 20,
 'time': 21,
 'to': 22,
 'travel': 23,
 'trot': 24,
 'turkey': 25,
 'turtle': 26}

In [ ]:
#Swap word for id in each document
document = [list(map(lambda x: vocab[x], doc)) for doc in rawdocs]

In [ ]:
document

[[7, 25, 14, 25, 6, 8],
 [9, 11, 22, 7, 4, 14, 8],
 [25, 24, 15, 14, 19, 8],
 [16, 15, 20, 26],
 [21, 23, 17, 15],
 [12, 14, 19],
 [12, 3, 0, 1, 17, 13, 10, 5, 12],
 [2, 12, 18]]

### Create the topic-word matrix

In [ ]:
def initialiseWordTopicMatrix(vocab, document, K):
    #Initialise the word-topic count matrix
    TopicWordMatrix = np.zeros((K, len(vocab)))
    #Randomly assign topic for each word in each document
    topicAssignmentList = [[random.randint(0,K-1) for i in range(len(doc))] for doc in document]

    for iDoc, doc in enumerate(document): #For all document
        for iToken, wordId in enumerate(doc): #For all token
            #Find the topic of the given token
            tokenTopic =  topicAssignmentList[iDoc][iToken]
            #Update the wordTopicMatrix
            TopicWordMatrix[tokenTopic][wordId] += 1
    return TopicWordMatrix, topicAssignmentList

In [ ]:
TopicWordMatrix, topicAssignmentList = initialiseWordTopicMatrix(vocab, document, K)

In [ ]:
TopicWordMatrix

array([[1., 1., 1., 0., 1., 1., 1., 1., 1., 0., 0., 0., 2., 1., 3., 1.,
        1., 2., 0., 1., 0., 0., 0., 0., 1., 1., 1.],
       [0., 0., 0., 1., 0., 0., 0., 1., 2., 1., 1., 1., 2., 0., 1., 2.,
        0., 0., 1., 1., 1., 1., 1., 1., 0., 2., 0.]])

In [ ]:
topicAssignmentList

[[1, 1, 1, 1, 0, 1],
 [1, 1, 1, 0, 0, 0, 0],
 [0, 0, 1, 0, 1, 1],
 [0, 1, 1, 0],
 [1, 1, 0, 0],
 [0, 0, 0],
 [1, 1, 0, 0, 0, 0, 1, 0, 1],
 [0, 0, 1]]

### Create the document-topic matrix 

In [ ]:
def initialiseDocumentTopicMatrix(topicAssignmentList, document):
    documentTopicMatrix = np.zeros((len(document), K))
    for iDoc in range(len(document)):
        for iTopic in range(K):
            #Update document matrix topic according to topicAssignmentList
            documentTopicMatrix[iDoc][iTopic] = topicAssignmentList[iDoc].count(iTopic)
    return documentTopicMatrix

In [ ]:
documentTopicMatrix = initialiseDocumentTopicMatrix(topicAssignmentList, document)

In [ ]:
documentTopicMatrix

array([[1., 5.],
       [4., 3.],
       [3., 3.],
       [2., 2.],
       [2., 2.],
       [3., 0.],
       [5., 4.],
       [2., 1.]])

### LDA iterations 

In [ ]:
def ldaModel(K, alpha, eta, iterationNb, document, vocab, TopicWordMatrix, topicAssignmentList, documentTopicMatrix):
    #For each iteration
    for i in range(iterationNb):
        #For each document
        for iDoc, doc in enumerate(document):
            #For each word in the document
            for iToken, wordId in enumerate(doc):
                #Initial topic for the token
                oldTopic = topicAssignmentList[iDoc][iToken]

                #Focus of the i-th Token - decrement in the matrices
                documentTopicMatrix[iDoc][oldTopic] -= 1
                TopicWordMatrix[oldTopic][wordId] -= 1
                
                #Gibbs-Sampling
                weight = []
                for iTopic in range(K):
                    #A term
                    num_a = topicAssignmentList[iDoc].count(iTopic)+alpha
                    denom_a = len(vocab)-1+alpha
                    #B term
                    num_b = TopicWordMatrix[iTopic][wordId] + eta
                    denom_b = TopicWordMatrix.sum(axis=0)[K]+eta
                    #Proba
                    weight.append((num_a/denom_a)*(num_b/denom_b))
                
                #Draw topic - multinomial distribution
                newTopic = random.choices(range(K), weights = weight, k = 1)[0]
                #Re-assign topic
                documentTopicMatrix[iDoc][newTopic] += 1
                TopicWordMatrix[newTopic][wordId] += 1
                topicAssignmentList[iDoc][iToken] = newTopic
    #Normalize matrix
    documentTopicMatrix = ((documentTopicMatrix+alpha).T/(documentTopicMatrix+alpha).sum(axis=1)).T
    TopicWordMatrix = ((TopicWordMatrix+alpha).T/(TopicWordMatrix+alpha).sum(axis=1)).T
    return documentTopicMatrix, TopicWordMatrix, topicAssignmentList


In [ ]:
documentTopicMatrixUpdate, TopicWordMatrixUpdate, topicAssignmentListUpdate = ldaModel(K, alpha, eta, iterationNb, document, vocab, TopicWordMatrix, topicAssignmentList, documentTopicMatrix)


In [ ]:
documentTopicMatrixUpdate

array([[0.01612903, 0.98387097],
       [0.01388889, 0.98611111],
       [0.01612903, 0.98387097],
       [0.02380952, 0.97619048],
       [0.73809524, 0.26190476],
       [0.03125   , 0.96875   ],
       [0.98913043, 0.01086957],
       [0.03125   , 0.96875   ]])

In [ ]:
TopicWordMatrixUpdate

array([[0.07482993, 0.07482993, 0.00680272, 0.07482993, 0.00680272,
        0.07482993, 0.00680272, 0.00680272, 0.00680272, 0.00680272,
        0.07482993, 0.00680272, 0.14285714, 0.07482993, 0.00680272,
        0.00680272, 0.00680272, 0.14285714, 0.00680272, 0.00680272,
        0.00680272, 0.07482993, 0.00680272, 0.07482993, 0.00680272,
        0.00680272, 0.00680272],
       [0.0030581 , 0.0030581 , 0.03363914, 0.0030581 , 0.03363914,
        0.0030581 , 0.03363914, 0.06422018, 0.09480122, 0.03363914,
        0.0030581 , 0.03363914, 0.06422018, 0.0030581 , 0.12538226,
        0.09480122, 0.03363914, 0.0030581 , 0.03363914, 0.06422018,
        0.03363914, 0.0030581 , 0.03363914, 0.0030581 , 0.03363914,
        0.09480122, 0.03363914]])

In [ ]:
topicAssignmentListUpdate

[[1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1],
 [0, 0, 0, 1],
 [1, 1, 1],
 [0, 0, 0, 0, 0, 0, 0, 0, 0],
 [1, 1, 1]]

### Show topic 

In [ ]:
def displayTopic(TopicWordMatrixUpdate, vocab, nb_word):
    vocab = {v: k for k, v in vocab.items()} #Swap id and value to have a dict {id: "word"}
    for topicNb, wordPerTopic in enumerate(TopicWordMatrixUpdate):
        print(f"\n>>> Topic {topicNb}")
        TopicWordMatrixSeries = pd.Series(wordPerTopic).sort_values(ascending=False) 
        wordIds = TopicWordMatrixSeries.index
        topicToString = []
        for i in range(nb_word):
            topicToString.append(f"{vocab[wordIds[i]]}*{round(TopicWordMatrixSeries[wordIds[i]],2)}")
        print('+'.join(topicToString))

In [ ]:
displayTopic(TopicWordMatrixUpdate, vocab, 6)


>>> Topic 0
space*0.14+movie*0.14+air*0.07+is*0.07+travel*0.07+time*0.07

>>> Topic 1
on*0.13+turkey*0.09+race*0.09+holiday*0.09+thanksgiving*0.06+eat*0.06
